This notebook was run in colab as I had issues at my local machine

In [ ]:
!pip install simpletransformers

In [ ]:
!pip install modelbit

In [ ]:
!unzip bert_base_no_batch_split.zip

Archive:  bert_base_no_batch_split.zip
   creating: bert_base_no_batch_split/
   creating: bert_base_no_batch_split/best_model/
  inflating: bert_base_no_batch_split/best_model/config.json  
  inflating: bert_base_no_batch_split/best_model/eval_results.txt  
  inflating: bert_base_no_batch_split/best_model/model.safetensors  
  inflating: bert_base_no_batch_split/best_model/model_args.json  
  inflating: bert_base_no_batch_split/best_model/optimizer.pt  
  inflating: bert_base_no_batch_split/best_model/scheduler.pt  
  inflating: bert_base_no_batch_split/best_model/special_tokens_map.json  
  inflating: bert_base_no_batch_split/best_model/tokenizer_config.json  
  inflating: bert_base_no_batch_split/best_model/training_args.bin  
  inflating: bert_base_no_batch_split/best_model/vocab.txt  
  inflating: bert_base_no_batch_split/config.json  
  inflating: bert_base_no_batch_split/model.safetensors  
  inflating: bert_base_no_batch_split/model_args.json  
  inflating: bert_base_no_batch_s

In [ ]:
from uuid import uuid4

import modelbit
from simpletransformers.question_answering import QuestionAnsweringModel

In [21]:
model = QuestionAnsweringModel("bert",
                                "/content/bert_base_no_batch_split",
                                use_cuda=False)


def predict_qa_model(question: str, context: str):
    data = [{'context': context, 'qas': [{'id': 1, 'question': question}]}]
    return model.predict(data, n_best_size=3)[0]


In [ ]:
modelbit.login()

<module 'modelbit' from '/usr/local/lib/python3.10/dist-packages/modelbit/__init__.py'>

In [22]:
modelbit.deploy(predict_qa_model, python_packages=['torch==2.0.1+cu117', 'simpletransformers==0.70.1'])

Encrypting 'model': 100%|██████████| 412M/412M [00:06<00:00, 64.0MB/s]
Uploading 'model': 100%|██████████| 381M/381M [00:05<00:00, 75.1MB/s]


In [23]:
!curl -s -XPOST "https://boglis002.app.modelbit.com/v1/predict_qa_model/latest" -d '{"data": ["What was the cost to society?", "Other legislation followed, including the Migratory Bird Conservation Act of 1929, a 1937 treaty prohibiting the hunting of right and gray whales, and the Bald Eagle Protection Act of 1940. These later laws had a low cost to society—the species were relatively rare—and little opposition was raised"]}' | json_pp

{
   "data" : [
      {
         "answer" : [
            "low",
            "low cost to society—the species were relatively rare"
         ],
         "id" : 1
      }
   ]
}
